<a href="https://colab.research.google.com/github/Gooogr/Keras_Fast_Style_Transfer/blob/master/Vanilla_Style_Transfer_(with_L_BFGS)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19
from keras import backend as K

import numpy as np

from imageio import imwrite
import cv2 

import time

from scipy.optimize import fmin_l_bfgs_b

from matplotlib import pyplot as plt

from google.colab.patches import cv2_imshow   #instead of cv2.imshow
from google.colab import drive


Using TensorFlow backend.


[How to add data from google drive to collab file](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92)


In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
target_image_path = '/content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/dogs.jpg'
style_reference_image_path = '/content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/night.jpg'
result_directory_path = '/content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/epochs_results'

In [0]:
## uncomment to check images

## you can use 'load_img(file_path)' instead of cv2

# img = cv2.imread(target_image_path)
# cv2_imshow(img)

# img = cv2.imread(style_reference_image_path)
# cv2_imshow(img)

In [0]:
# Scale input image to VGG19 size
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [0]:
# Вспомогательные функции
def preprocess_image(image_path):
    img = load_img(image_path, target_size = (img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):    
    x[:, :, 0] += 103.939 # Нулевое центрирование путем удаления среднего значения пиксела из ImageNet.                          
    x[:, :, 1] += 116.779 # Отменяет преобразование выполненное vgg19.preprocess_input
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1] #BGR -> RGB
    x = np.clip(x, 0, 255).astype('uint8')
    return x

Настроим сеть VGG19.
Она принимает на вход пакет из трех изображений:
* Изображение с образцом стиля
* Целевое изображение
* Заготовка, куда будет помещен результат

Изображения с образцом стиля и целью определяются как константы.

In [7]:
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor,
                    weights = 'imagenet',
                    include_top = False)

Определим функцию потери. Она характеризуется функциями потерь содержимого, стиля и общей потери вариации.

In [0]:
def content_loss(base, combination):
    '''
    Функция потерь содержимого.
    '''
    return K.sum(K.square(combination - base))

In [0]:
def gram_matrix(x):
    '''
    Вспомогательная функция. Вычисляет матрицу Грама для корреляционной матрицы.
    http://pmpu.ru/vf4/dets/gram
    '''
    # K.permute_dimensions - Permutes axes in a tensor. https://www.tensorflow.org/api_docs/python/tf/keras/backend/permute_dimensions
    # K.batch_flatten - Turn a nD tensor into a 2D tensor with same 0th dimension. https://www.tensorflow.org/api_docs/python/tf/keras/backend/batch_flatten
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    '''
    Функция потерь стиля
    '''
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [0]:
def total_variation_loss(x):
    '''
    Фунция общей потери вариации, стимулирует пространственную целостность итогового изображения, 
    позволяет избежать мозаичного эффекта.
    Можно интерпретировать как регуляризацию потерь.
    '''
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :]) # Сдвиг в 1 пиксель по высоте\ширине
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))    

Для вычисления потери содержимого используется только один верхний слой block5_conv2.<br>
Для вычисления потери стиля - нижние слои из каждого сверточного блока.<br>
Общая потеря вариации добавляется в конце.

In [11]:
# Определение общей потери
output_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025 # Чем больше - тем больше походит итоговое изображение на целевое

loss = K.variable(0.)

# content loss part
layer_features = output_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

# style loss part
for layer_name in style_layers:
    layer_features = output_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl

# variation loss part
loss += total_variation_weight * total_variation_loss(combination_image)

Настройка процесса градиентного спуска.

Градиентный спуск выполняется с помощью алгоритма L - BFGS, как в и оригинальной статье.
Этот алгоритм реализован в пакете SciPy, однако есть 2 нюанса:<br>
1) Алгоритм требует передачи значений функции потерь и градиентов в виде двух отдельных функций.<br>
2) Может применятся только к плоским векторам.

Для решения проблемы напишем класс-оберту, который будет вычислять значения потерь и градиентов одновременно, возвращать значение потерь при первом обращении и кэшировать значение градиентов при первом обращении.

In [12]:
grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Запускаем весь процесс

In [0]:
result_prefix = 'result'
iterations = 1000

x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime = evaluator.grads, maxfun = 20)
    print('Current loss value:', min_val)
    
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_directory_path + result_prefix + '_at_iteration_%d.png' % i
    imwrite(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d complited in %ds' % (i, end_time - start_time))    

Start of iteration 0
Current loss value: 3204929300.0
Image saved as /content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/epochs_resultsresult_at_iteration_0.png
Iteration 0 complited in 23s
Start of iteration 1
Current loss value: 1150597400.0
Image saved as /content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/epochs_resultsresult_at_iteration_1.png
Iteration 1 complited in 17s
Start of iteration 2
Current loss value: 632059650.0
Image saved as /content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/epochs_resultsresult_at_iteration_2.png
Iteration 2 complited in 17s
Start of iteration 3
Current loss value: 449259460.0
Image saved as /content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/epochs_resultsresult_at_iteration_3.png
Iteration 3 complited in 17s
Start of iteration 4
Current loss value: 321028500.0
Image saved as /content/drive/My Drive/colab_notebooks/Keras_Fast_Style_Transfer/img/epochs_resultsresult_at_iteratio